In [63]:
pip install yfinance riskfolio-lib


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import yfinance as yf
import riskfolio as rp
import pandas as pd
import numpy as np

# Fetch stock data
tickers = ['NVDA', 'SMCI', 'KNSL', 'GCT', 'PDD', 'CRWD', 'AVGO', 'SNOW']
start_date = '2021-01-01'
end_date = '2024-03-20'

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Resample data to monthly frequency
monthly_data = data.resample('M').last()

# Calculate returns
returns = monthly_data.pct_change().dropna()

# Create a portfolio object
port = rp.Portfolio(returns)

# Estimate expected returns and covariance matrix
port.assets_stats(method_mu='hist', method_cov='hist')

# Define the asset bounds and constraints
port.ainequality = None
port.binequality = None
port.lowerbound = 0.05
port.upperbound = 0.5
port.budget = 1

# Optimize the portfolio using ERP
model = 'Classic' # Could be 'Classic' (historical), 'BL' (Black Litterman), or 'FM' (Factor Model)
rm = 'MV' # Risk measure could be 'MV' (variance), 'MAD' (mean absolute deviation), or 'MSV' (semi-variance)
rf = 0.0 # Risk-free rate
b = None # Risk budgeting vector (None for ERP)

w = port.rp_optimization(model=model, rm=rm, rf=rf, b=b)

# Print the optimal weights
print("\nOptimal Weights:")
print(w.T)


[*********************100%%**********************]  8 of 8 completed



Optimal Weights:
             AVGO      CRWD       GCT      KNSL      NVDA      PDD      SMCI   
weights  0.151058  0.123807  0.051839  0.222632  0.085263  0.13649  0.046216  \

             SNOW  
weights  0.182696  
The problem doesn't have a solution with actual input parameters
The problem doesn't have a solution with actual input parameters
The problem doesn't have a solution with actual input parameters


ValueError: All objects passed were None

In [66]:
print("\nMonthly Stock Returns")
print(returns)


Monthly Stock Returns
Ticker          AVGO      CRWD       GCT      KNSL      NVDA       PDD   
Date                                                                     
2022-09-30 -0.102937 -0.097476 -0.309898  0.007256 -0.195534 -0.122300  \
2022-10-31  0.058805 -0.021904 -0.537092  0.233928  0.111871 -0.123841   
2022-11-30  0.172105 -0.270161  0.574786 -0.021669  0.254155  0.496261   
2022-12-31  0.023166 -0.105057 -0.227951 -0.151488 -0.136442 -0.005973   
2023-01-31  0.046286  0.005794  0.028119  0.064699  0.336869  0.201471   
2023-02-28  0.015863  0.139660 -0.135043  0.145098  0.188309 -0.104613   
2023-03-31  0.087276  0.137294  0.245059 -0.058205  0.196662 -0.134846   
2023-04-30 -0.023443 -0.125382 -0.122222  0.088489 -0.001008 -0.102108   
2023-05-31  0.289641  0.333861  0.220615 -0.072210  0.363436 -0.041526   
2023-06-30  0.079325 -0.082808  0.001482  0.235065  0.118210  0.058481   
2023-07-31  0.035991  0.100701  0.292899 -0.004196  0.104652  0.299103   
2023-08-31  0.0

In [67]:
monthly_return = returns.mean()
annual_return = returns.mean() * 12

monthly_volatility = returns.std()
annual_volatility = monthly_volatility * np.sqrt(12)

# Create a DataFrame to store volatility values
stock_df = pd.DataFrame({
  'Ret (M)': monthly_return,
  'Ret (Y)': annual_return,
  'Vol (M)': monthly_volatility,
  'Vol (Y)': annual_volatility})

# Custom function for formatting as percentage
def format_as_percent(value):
    return "{:.2f}%".format(value * 100)

# Apply the formatting function to all elements in the DataFrame
stock_df_formatted = stock_df.applymap(format_as_percent)

print("Stock:")
print(stock_df_formatted)

Stock:
       Ret (M)  Ret (Y) Vol (M)  Vol (Y)
Ticker                                  
AVGO     5.52%   66.30%   9.69%   33.56%
CRWD     4.04%   48.47%  14.99%   51.93%
GCT      9.67%  116.09%  36.79%  127.44%
KNSL     4.56%   54.76%  13.08%   45.32%
NVDA    10.95%  131.39%  15.89%   55.04%
PDD      4.56%   54.77%  18.97%   65.73%
SMCI    19.08%  229.00%  35.31%  122.32%
SNOW    -0.27%   -3.27%  10.03%   34.73%


In [68]:
correlation_matrix = returns.corr()

print("\nCorrelation Matrix:")
print(correlation_matrix)


Correlation Matrix:
Ticker      AVGO      CRWD       GCT      KNSL      NVDA       PDD      SMCI   
Ticker                                                                         
AVGO    1.000000  0.372404  0.643224 -0.059210  0.658663  0.246629  0.589000  \
CRWD    0.372404  1.000000  0.155561  0.006139  0.394240 -0.052034  0.513910   
GCT     0.643224  0.155561  1.000000 -0.041382  0.428351  0.312679  0.268934   
KNSL   -0.059210  0.006139 -0.041382  1.000000  0.346480 -0.167706  0.304194   
NVDA    0.658663  0.394240  0.428351  0.346480  1.000000  0.227832  0.663242   
PDD     0.246629 -0.052034  0.312679 -0.167706  0.227832  1.000000 -0.106545   
SMCI    0.589000  0.513910  0.268934  0.304194  0.663242 -0.106545  1.000000   
SNOW    0.429519  0.634140  0.072300 -0.016515  0.251165  0.310879  0.190598   

Ticker      SNOW  
Ticker            
AVGO    0.429519  
CRWD    0.634140  
GCT     0.072300  
KNSL   -0.016515  
NVDA    0.251165  
PDD     0.310879  
SMCI    0.190598  
SNOW   

In [69]:
port_mu = port.mu @ w  # Portfolio Expected Return
port_vol = np.sqrt(w.T @ port.cov @ w)  # Portfolio Standard Deviation (Volatility)

annual_mu = port_mu * 12
annual_vol = port_vol * np.sqrt(12)

portfolio = {
    'Exp Ret (M)': port_mu.iloc[0, 0],
    'Exp Ret (Y)': annual_mu.iloc[0, 0],
    'Exp Vol (M)': port_vol.iloc[0, 0],
    'Exp Vol (Y)': annual_vol.iloc[0, 0]
}

print("Portfolio:")
for key, value in portfolio.items():
    print(f"{key}:\t{value*100:.2f}%")

Portfolio:
Exp Ret (M):	4.94%
Exp Ret (Y):	59.28%
Exp Vol (M):	8.72%
Exp Vol (Y):	30.22%
